In [1]:
!pip install google-play-scraper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 1.7 MB/s eta 0:00:00


In [2]:
!pip install sastrawi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 3.7 MB/s eta 0:00:00


In [3]:
# mengimport pustaka google_play_scraper untuk mengakses ulasan dan dan informasi aplikasi dari Google Play Store.
from google_play_scraper import app, reviews, Sort, reviews_all

import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
seed = 0
np.random.seed(seed)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score

import datetime as dt
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

from wordcloud import WordCloud

import nltk
nltk.download('punkt_tab')
nltk.download('stopwords')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
# mengambil semua ulasan dari aplikasi dengan ID 'id.bni.wondr' di play store

scrapreview = reviews_all(
    'id.bni.wondr',
    lang='id',
    country='id',
    sort=Sort.MOST_RELEVANT,
    count=1000
)

In [5]:
# Memubuat DataFrame dari hasil scrapreview
reviews_wondr = pd.DataFrame(scrapreview)
reviews_wondr.to_csv('ulasan_apk_wondr.csv', index=False)

In [6]:
# menghitung jumlah baris dan kolom dalam DataFrame
jumlah_ulasan, jumlah_kolom = reviews_wondr.shape
reviews_wondr.shape

(21425, 11)

In [7]:
reviews_wondr.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
0,1d233a1c-c9e4-4aa4-906a-f12db8ebc4dc,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Sampai saat ini masih oke sih. Tampilannya sim...,4,22,1.2.0,2025-02-05 07:07:41,"Hai Kak Rima Z, mohon maaf atas ketidaknyamana...",2025-02-05 07:26:27,1.2.0
1,d1e5113c-b239-4c23-b0e7-822b092e76ea,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Minta tolong dikembangkan lagi untuk informasi...,4,5,1.2.0,2025-02-09 11:06:43,"Hai Kak Hafidz, maaf atas kendalanya. Ada ya K...",2025-02-10 01:18:23,1.2.0
2,4710aa01-318d-4122-b87e-5ac60ba8a0dd,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Aplikasi sampah ini, bagusan yang dulu aja. Ma...",1,86,1.2.0,2025-02-10 12:05:37,"Hai Kak Doffa Rizdhia, maaf utk kendalanya. Pa...",2025-02-10 12:32:57,1.2.0
3,75c9e1a6-b35c-4cfd-a78f-691bde7c16cb,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Di aplikasi wonder ini tidak ada daftar favori...,2,50,1.2.0,2025-02-03 01:10:37,"Hai Kak Mba Nely, maaf atas kendalanya. Kalau ...",2025-02-03 01:24:30,1.2.0
4,1c309cd4-8f95-496a-9035-cf432c97ed34,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Terlalu banyak pengembangan & membuang anggara...,1,89,1.2.0,2025-01-26 08:24:37,"Hai Kak Yuvensius Vega, maaf atas ketidaknyama...",2025-01-26 08:44:28,1.2.0


In [8]:
reviews_wondr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21425 entries, 0 to 21424
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   reviewId              21425 non-null  object        
 1   userName              21425 non-null  object        
 2   userImage             21425 non-null  object        
 3   content               21425 non-null  object        
 4   score                 21425 non-null  int64         
 5   thumbsUpCount         21425 non-null  int64         
 6   reviewCreatedVersion  18351 non-null  object        
 7   at                    21425 non-null  datetime64[ns]
 8   replyContent          15457 non-null  object        
 9   repliedAt             15457 non-null  datetime64[ns]
 10  appVersion            18351 non-null  object        
dtypes: datetime64[ns](2), int64(2), object(7)
memory usage: 1.8+ MB


In [9]:
# Menghapus baris yang memiliki missing value
reviews_wondr_clean = reviews_wondr.dropna()
# Menghapus baris duplikat
reviews_wondr_clean = reviews_wondr_clean.drop_duplicates()

In [10]:
reviews_wondr_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12921 entries, 0 to 20488
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   reviewId              12921 non-null  object        
 1   userName              12921 non-null  object        
 2   userImage             12921 non-null  object        
 3   content               12921 non-null  object        
 4   score                 12921 non-null  int64         
 5   thumbsUpCount         12921 non-null  int64         
 6   reviewCreatedVersion  12921 non-null  object        
 7   at                    12921 non-null  datetime64[ns]
 8   replyContent          12921 non-null  object        
 9   repliedAt             12921 non-null  datetime64[ns]
 10  appVersion            12921 non-null  object        
dtypes: datetime64[ns](2), int64(2), object(7)
memory usage: 1.2+ MB


In [11]:
def cleaningText(text):
    text = re.sub(r'@[A-Za-z0-9]+', '', text) # menghapus mention
    text = re.sub(r'#[A-Za-z0-9]+', '', text) # menghapus hastag
    text = re.sub(r'RT[\s]', '', text) # menghapus RT
    text = re.sub(r"http\S+", '', text) # menghapus link
    text = re.sub(r'[0-9]', '', text) # menghapus angka
    text = re.sub(r'[^\w\s]', '', text) # menghapus karakter selain huruf dan angka

    text = text.replace('\n', ' ') # mengganti garis baru dengan spasi
    text = text.translate(str.maketrans('', '', string.punctuation)) # menghapus semua tanda baca
    text = text.strip(' ') # mengahapus spasi dari kanan kiri teks
    return text

def casefoldingText(text): # mengubah semua karakter dalam teks menjadi lower case
    text = text.lower()
    return text

def tokenizingText(text): # memecah atau membagi string, teks menjadi daftar token
    text = word_tokenize(text)
    return text

def filteringText(text): # Menghapus stopwords dalam teks
    listStopwords = set(stopwords.words('indonesian'))
    listStopwords1 = set(stopwords.words('english'))
    listStopwords.update(listStopwords1)
    listStopwords.update(['iya', 'yaa', 'gak', 'nya', 'na', 'sih', 'ku', 'di', 'ga', 'ya', 'gaa', 'loh', 'kah', 'woi', 'woii', 'woy'])
    filtered = []
    for txt in text:
        if txt not in listStopwords:
            filtered.append(txt)
    text = filtered
    return text

def stemmingText(text): # mengurangi kata ke bentuk dasranya
    # membuat objek stemmer
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()

    # memecah teks menjadi daftar kata
    words = text.split()

    # menerapkan stemming pada setiap kata dalam daftar
    stemmed_words = [stemmer.stem(word) for word in words]

    # menggabungkan kata-kata yang telah distem
    stemmed_text = ' '.join(stemmed_words)

    return stemmed_text

def toSentence(list_words): # mengubah daftar kata menjadi kalimat
    sentence = ' '.join(word for word in list_words)
    return sentence

In [12]:
slangwords = {"@": "di", "abis": "habis", "wtb": "beli", "masi": "masih", "wts": "jual", "wtt": "tukar", "bgt": "banget", "maks": "maksimal"}
def fix_slangwords(text):
    words = text.split()
    fixed_words = []

    for word in words:
        if word.lower() in slangwords:
            fixed_words.append(slangwords[word.lower()])
        else:
            fixed_words.append(word)

    fixed_text = ' '.join(fixed_words)
    return fixed_text

In [13]:
# Membersihkan teks
reviews_wondr_clean['text_clean'] = reviews_wondr_clean['content'].apply(cleaningText)

# Mengubah huruf dalam teks menjadi huruf kecil
reviews_wondr_clean['text_casefoldingText'] = reviews_wondr_clean['text_clean'].apply(casefoldingText)

# mengganti kata-kata slan dengan kata-kata formal
reviews_wondr_clean['text_slangwords'] = reviews_wondr_clean['text_casefoldingText'].apply(fix_slangwords)

# memecah teks menjadi token
reviews_wondr_clean['text_tokenizingText'] = reviews_wondr_clean['text_slangwords'].apply(tokenizingText)

# Menghapus kata-kata stop (kata-kata umum)
reviews_wondr_clean['text_stopword'] = reviews_wondr_clean['text_tokenizingText'].apply(filteringText)

# Menggabungkan token-token menjadi kalimat
reviews_wondr_clean['text_akhir'] = reviews_wondr_clean['text_stopword'].apply(toSentence)

In [14]:
reviews_wondr_clean.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion,text_clean,text_casefoldingText,text_slangwords,text_tokenizingText,text_stopword,text_akhir
0,1d233a1c-c9e4-4aa4-906a-f12db8ebc4dc,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Sampai saat ini masih oke sih. Tampilannya sim...,4,22,1.2.0,2025-02-05 07:07:41,"Hai Kak Rima Z, mohon maaf atas ketidaknyamana...",2025-02-05 07:26:27,1.2.0,Sampai saat ini masih oke sih Tampilannya simp...,sampai saat ini masih oke sih tampilannya simp...,sampai saat ini masih oke sih tampilannya simp...,"[sampai, saat, ini, masih, oke, sih, tampilann...","[oke, tampilannya, simple, mudah, dipahami, be...",oke tampilannya simple mudah dipahami bedanya ...
1,d1e5113c-b239-4c23-b0e7-822b092e76ea,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Minta tolong dikembangkan lagi untuk informasi...,4,5,1.2.0,2025-02-09 11:06:43,"Hai Kak Hafidz, maaf atas kendalanya. Ada ya K...",2025-02-10 01:18:23,1.2.0,Minta tolong dikembangkan lagi untuk informasi...,minta tolong dikembangkan lagi untuk informasi...,minta tolong dikembangkan lagi untuk informasi...,"[minta, tolong, dikembangkan, lagi, untuk, inf...","[tolong, dikembangkan, informasi, notifikasi, ...",tolong dikembangkan informasi notifikasi trans...
2,4710aa01-318d-4122-b87e-5ac60ba8a0dd,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Aplikasi sampah ini, bagusan yang dulu aja. Ma...",1,86,1.2.0,2025-02-10 12:05:37,"Hai Kak Doffa Rizdhia, maaf utk kendalanya. Pa...",2025-02-10 12:32:57,1.2.0,Aplikasi sampah ini bagusan yang dulu aja Masa...,aplikasi sampah ini bagusan yang dulu aja masa...,aplikasi sampah ini bagusan yang dulu aja masa...,"[aplikasi, sampah, ini, bagusan, yang, dulu, a...","[aplikasi, sampah, bagusan, aja, face, recogni...",aplikasi sampah bagusan aja face recognitionny...
3,75c9e1a6-b35c-4cfd-a78f-691bde7c16cb,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Di aplikasi wonder ini tidak ada daftar favori...,2,50,1.2.0,2025-02-03 01:10:37,"Hai Kak Mba Nely, maaf atas kendalanya. Kalau ...",2025-02-03 01:24:30,1.2.0,Di aplikasi wonder ini tidak ada daftar favori...,di aplikasi wonder ini tidak ada daftar favori...,di aplikasi wonder ini tidak ada daftar favori...,"[di, aplikasi, wonder, ini, tidak, ada, daftar...","[aplikasi, wonder, daftar, favorit, transaksi,...",aplikasi wonder daftar favorit transaksi pengi...
4,1c309cd4-8f95-496a-9035-cf432c97ed34,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Terlalu banyak pengembangan & membuang anggara...,1,89,1.2.0,2025-01-26 08:24:37,"Hai Kak Yuvensius Vega, maaf atas ketidaknyama...",2025-01-26 08:44:28,1.2.0,Terlalu banyak pengembangan membuang anggaran...,terlalu banyak pengembangan membuang anggaran...,terlalu banyak pengembangan membuang anggaran ...,"[terlalu, banyak, pengembangan, membuang, angg...","[pengembangan, membuang, anggaran, perusahaan,...",pengembangan membuang anggaran perusahaan bni ...


In [15]:
import csv
import requests
from io import StringIO

# membaca data kamus kata-kata positif dari github
lexicon_positive = dict()

# mengirim permintaan HTTP untuk mendapatakan file csv dari github
response = requests.get('https://raw.githubusercontent.com/angelmetanosaa/dataset/main/lexicon_positive.csv')

if response.status_code == 200:
    # jika berhasil
    reader = csv.reader(StringIO(response.text), delimiter=',')
    # Membaca teks respons sebagai file CSV menggunakan pembaca CSV dengan pemisah koma

    for row in reader:
        # mengulangi setiap baris dalam file CSV
        lexicon_positive[row[0]] = int(row[1])
        # menambahkan kata-kata positif dan skornya ke dalam kamus lexicon_positive

else:
    print("Failed to fetch positive lexicon data")

# membaca data kamus kata-kata negatif dari Github
lexicon_negative = dict()

response = requests.get('https://raw.githubusercontent.com/angelmetanosaa/dataset/main/lexicon_negative.csv')

if response.status_code == 200:
    # Jika permintaan berhasil
    reader = csv.reader(StringIO(response.text), delimiter=',')
    # Membaca teks respons sebagai file CSV menggunakan pembaca CSV dengan pemisah koma

    for row in reader:
        # Mengulangi setiap baris dalam file CSV
        lexicon_negative[row[0]] = int(row[1])
        # Menambahkan kata-kata negatif dan skornya dalam kamus lexicon_negative
else:
    print("Failed to fetch negative lexicon data")

In [16]:
# fungsi untuk menentukan polaritas sentimen dari tweet

def sentiment_anaysis_lexicon_indonesia(text):
    score = 0
    # inisialisasi skor sentimen ke 0

    for word in text:
        # mengulangi setiap kata dalam teks

        if (word in lexicon_positive):
            score = score + lexicon_positive[word]
            # jika kata dalam kamus positif, tambahkan skornya ke skor sentimen

    for word in text:
        # Mengulangi setiap kata dalam teks (sekali lagi)

        if (word in lexicon_negative):
            score = score + lexicon_negative[word]
            # Jika kata ada dalam kamus negatif, kurangkan skornya dari skor sentimen

    polarity=''
    # inisialisasi variabel polaritas

    if (score >= 0):
        polarity = 'positive'

    elif (score < 0):
        polarity = 'negative'

    return score, polarity
    # mengembalikan skor sentimen dan polaritas teks

In [17]:
results = reviews_wondr_clean['text_stopword'].apply(sentiment_anaysis_lexicon_indonesia)
results = list(zip(*results))
reviews_wondr_clean['polarity_score'] = results[0]
reviews_wondr_clean['polarity'] = results[1]
print(reviews_wondr_clean['polarity'].value_counts())

polarity
positive    6647
negative    6274
Name: count, dtype: int64


In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

# pisahkan data menjadi fitur (tweet) dan label (sentimen)
X = reviews_wondr_clean['text_akhir']
y = reviews_wondr_clean['polarity']

# ekstrasi fitur dengan TF-IDF
tfidf = TfidfVectorizer(max_features=200, min_df=17, max_df=0.8)
X_tfidf = tfidf.fit_transform(X)

# konversi hasil ekstraksi fitur menjadi dataframe
features_df = pd.DataFrame(X_tfidf.toarray(), columns=tfidf.get_feature_names_out())

# menampilkan hasil ekstraksi fitur
features_df


,aja,akun,aman,aneh,apk,aplikasi,aplikasinya,app,atm,bagus,...,uang,udah,udh,ulang,update,verifikasi,wajah,wonder,wondr,yg
0,0.000000,0.000000,0.0,0.254526,0.0,0.000000,0.0,0.470212,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000
1,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.338744,0.0,0.0,0.000000,0.0,0.000000,0.0,0.288725,0.0,0.238902
2,0.453771,0.000000,0.0,0.000000,0.0,0.151675,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.222483,0.0,0.000000,0.0,0.000000
3,0.000000,0.000000,0.0,0.000000,0.0,0.139005,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.275706,0.0,0.000000,0.0,0.233120,0.0,0.000000
4,0.000000,0.248656,0.0,0.000000,0.0,0.113394,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.157353
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12916,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000
12917,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000
12918,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,1.0,...,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000
12919,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000


In [25]:
# bagi data menjadi data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

Naive Bayes

In [26]:
from sklearn.naive_bayes import BernoulliNB

naive_bayes = BernoulliNB()
naive_bayes.fit(X_train.toarray(), y_train)

# Prediksi sentimen pada data pelatihan dan uji
y_pred_train_nb = naive_bayes.predict(X_train.toarray())
y_pred_test_nb = naive_bayes.predict(X_test.toarray())

# Evaluasi akurasi model
accuracy_train_nb = accuracy_score(y_pred_train_nb, y_train)
accuracy_test_nb = accuracy_score(y_pred_test_nb, y_test)

print("Naive Bayes - accuracy_train:", accuracy_train_nb)
print("Naive Bayes - accuracy_test:", accuracy_test_nb)

Naive Bayes - accuracy_train: 0.7977941176470589
Naive Bayes - accuracy_test: 0.8104448742746615


Random Forest

In [30]:
from sklearn.ensemble import RandomForestClassifier

random_forest = RandomForestClassifier()
random_forest.fit(X_train.toarray(), y_train)

# Prediksi sentimen pada data pelatihan dan uji
y_pred_train_rf = random_forest.predict(X_train.toarray())
y_pred_test_rf = random_forest.predict(X_test.toarray())

# Evaluasi akurasi model
accuracy_train_rf = accuracy_score(y_pred_train_rf, y_train)
accuracy_test_rf = accuracy_score(y_pred_test_rf, y_test)

print("Random Forest - accuracy_train:", accuracy_train_rf)
print("Random Forest - accuracy_test:", accuracy_test_rf)

Random Forest - accuracy_train: 0.9822948916408669
Random Forest - accuracy_test: 0.8715667311411992


Logistic Regression

In [31]:
from sklearn.linear_model import LogisticRegression

logistic_regression = LogisticRegression()
logistic_regression.fit(X_train.toarray(), y_train)

# Prediksi sentimen pada data pelatihan dan uji
y_pred_train_lr = logistic_regression.predict(X_train.toarray())
y_pred_test_lr = logistic_regression.predict(X_test.toarray())

# Evaluasi akurasi model
accuracy_train_lr = accuracy_score(y_pred_train_lr, y_train)
accuracy_test_lr = accuracy_score(y_pred_test_lr, y_test)

print("Logistic Regression - accuracy_train:", accuracy_train_lr)
print("Logistic Regression - accuracy_test:", accuracy_test_lr)

Logistic Regression - accuracy_train: 0.8951238390092879
Logistic Regression - accuracy_test: 0.8928433268858801


Decision Tree

In [38]:
from sklearn.tree import DecisionTreeClassifier

decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train.toarray(), y_train)

# Prediksi sentimen pada data pelatihan dan uji
y_pred_train_dt = decision_tree.predict(X_train.toarray())
y_pred_test_dt = decision_tree.predict(X_test.toarray())

# Evaluasi akurasi model
accuracy_train_dt = accuracy_score(y_pred_train_dt, y_train)
accuracy_test_dt = accuracy_score(y_pred_test_dt, y_test)

print("Decision Tree - accuracy_train:", accuracy_train_dt)
print("Decision Tree - accuracy_test:", accuracy_test_dt)

Decision Tree - accuracy_train: 0.9822948916408669
Decision Tree - accuracy_test: 0.8224371373307543
